In [2]:
import pandas as pd 
import numpy as np
import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


In [3]:
metadata = pd.read_csv("../Project-Disney-World-DSF/metadata.csv")
hauntedhouse = pd.read_csv("../Project-Disney-World-DSF/haunted_mansion[87].csv")



**Preparing Hounted House Data**

In [4]:
#compute percentage of na-values per column; result: 96.59% of actual waiting times are na
hauntedhouse.isna().mean()

#extract and drop SACTMIN column from Df
act_times = hauntedhouse.copy()["SACTMIN"]
hauntedhouse = hauntedhouse.drop(["SACTMIN"], axis=1)

#format column date
hauntedhouse['date'] = pd.to_datetime(hauntedhouse['date'])
#format column datetime
hauntedhouse['datetime'] = pd.to_datetime(hauntedhouse['datetime'])

#drop rows with missing waiting times (coded as -999); dataset does NOT include observations during covid-19 closure time in the first place
hauntedhouse.loc[hauntedhouse['SPOSTMIN'] == -999,'SPOSTMIN'] = np.nan
hauntedhouse = hauntedhouse.dropna(axis=0, subset="SPOSTMIN")
hauntedhouse = hauntedhouse.reset_index(drop=True)  


**Preparing Metadata**

In [5]:
#looking at percentage of na-values per column
#metadata.isna().mean().sort_values(ascending=False).head(20)

#remove variables related to Hollywood Studios Park in California (and not Walt Disney World in Florida)
metadata.columns.str.startswith('HS').sum() 
metadata.columns.str.endswith('_HS').sum() 

metadata = metadata.loc[:, ~metadata.columns.str.startswith('HS')]
metadata = metadata.loc[:, ~metadata.columns.str.endswith('_HS')]

#filter out columns with all na-values
metadata.dropna(axis=1, how='all', inplace=True)

#format date
metadata['DATE'] = pd.to_datetime(metadata['DATE'])

#function that deals with string percentage values for columns that contain percentage of schools in session
def str_percent_to_float(dataframe):
    for col in dataframe.columns:
        if col.lower().startswith('insession'):
            dataframe[col] = dataframe[col].str.rstrip("%").astype(float)/100
            
str_percent_to_float(metadata)

#function that deals with string times of form '9:00' or '25:00'; converts to hours since midnight (float) for consistency and usability
sww = ["MKOPEN", "MKCLOSE", "MKEMHOPEN", "MKEMHCLOSE", "MKOPENYEST", "MKCLOSEYEST", "MKOPENTOM", "MKCLOSETOM", "EPOPEN", "EPCLOSE", "EPEMHOPEN",
"EPEMHCLOSE", "EPOPENYEST", "EPCLOSEYEST", "EPOPENTOM", "EPCLOSETOM", "AKOPEN", "AKCLOSE", "AKEMHOPEN", "AKEMHCLOSE", "AKOPENYEST", "AKCLOSEYEST",
"AKOPENTOM", "AKCLOSETOM", "MKPRDDT1", "MKPRDDT2", "MKPRDNT1", "MKPRDNT2", "MKFIRET1", "MKFIRET2", "EPFIRET1", "EPFIRET2", "AKSHWNT1", "AKSHWNT2"]

for col in sww:
    metadata[col].fillna("99", inplace=True)  #to indicate outliers

metadata["MKCLOSE"][0]

def format_times(x):
    if len(x)==4:
        time = '0'+ x
    elif len(x)==5 and x > '24:00':
        hour = int(x[:2])-24
        minute = x[-2:]
        time = '0' + str(hour) + ':' + minute
    elif x == '24:00':
        time = '00:00'
    else:
        time = x
    return time

def str_times_to_numerical(dataframe):
    for col in sww:
        dataframe[col] = dataframe[col].apply(format_times)
        dataframe[col] = dataframe[col].apply(lambda y: y.rstrip(':'))
        dataframe[col] = dataframe[col].apply(lambda x: (float(x[:2])+(float(x[-2:])/60)) if x[0] != 0 else (float(x[1])+(float(x[-2:])/60)))

str_times_to_numerical(metadata)

#one-hot encoding of categorical features
categorical_features = ["WDW_TICKET_SEASON", "SEASON", "HOLIDAYN", "WDWTICKETSEASON", "WDWRaceN", "WDWeventN", "WDWSEASON", "MKeventN", "EPeventN", "AKeventN", "HOLIDAYJ", "MKPRDDN", "MKPRDNN", "MKFIREN", "EPFIREN", "AKSHWNN"]

transformer = make_column_transformer(
    (OneHotEncoder(), categorical_features),
    remainder='passthrough')

transformed = transformer.fit_transform(metadata)
encoded_metadata = pd.DataFrame(transformed, columns=transformer.get_feature_names())
#new name of columns for i encoded column: 'onehotencoder__xi_oldcategoryname'

#function for filling missing values, to be used at later time
def imputation(dataframe):
        for col in dataframe.columns:
                dataframe[col] = dataframe[col].fillna(method='bfill')
                dataframe[col] = dataframe[col].fillna(dataframe[col].median())
        return dataframe


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Merge datasets

In [6]:
#change name of "DATE" column in metadata to fit with Haunted House 
encoded_metadata.rename(columns={"DATE":"date"}, inplace=True)

#merge metadata and waiting time data 
waittimes = pd.merge(hauntedhouse, encoded_metadata, how='left', on='date')

#Formatting of dates and times 
#drop date column due to redundancy
waittimes = waittimes.loc[:, waittimes.columns != "date"]

#create two new variables for hour and minute (day, month and year already included)
waittimes["HOUROFDAY"] = waittimes.copy()['datetime'].dt.hour
waittimes["MINUTEOFHOUR"] = waittimes.copy()['datetime'].dt.minute
#then drop datetime column
waittimes = waittimes.loc[:, waittimes.columns != 'datetime']


In [12]:
#subset for years 2019-2021 due to computing constraints
waittimes = waittimes.loc[waittimes["YEAR"].isin(range(2019,2022))].reset_index(drop=True)
waittimes

,SPOSTMIN,onehotencoder__x0_peak,onehotencoder__x0_regular,onehotencoder__x0_value,onehotencoder__x0_nan,onehotencoder__x1_CHRISTMAS,onehotencoder__x1_CHRISTMAS PEAK,onehotencoder__x1_COLUMBUS DAY,onehotencoder__x1_EASTER,onehotencoder__x1_FALL,...,MKFIRET2,EPFIREWK,EPFIRET1,EPFIRET2,AKPRDDAY,AKSHWNGT,AKSHWNT1,AKSHWNT2,HOUROFDAY,MINUTEOFHOUR
0,15.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,100.65,1.0,21.0,100.65,0.0,2.0,18.50,20.00,9,7
1,15.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,100.65,1.0,21.0,100.65,0.0,2.0,18.50,20.00,9,11
2,15.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,100.65,1.0,21.0,100.65,0.0,2.0,18.50,20.00,9,14
3,15.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,100.65,1.0,21.0,100.65,0.0,2.0,18.50,20.00,9,21
4,10.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,100.65,1.0,21.0,100.65,0.0,2.0,18.50,20.00,9,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85683,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,100.65,1.0,21.0,100.65,0.0,0.0,100.65,100.65,20,33
85684,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,100.65,1.0,21.0,100.65,0.0,0.0,100.65,100.65,20,40
85685,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,100.65,1.0,21.0,100.65,0.0,0.0,100.65,100.65,20,47
85686,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,100.65,1.0,21.0,100.65,0.0,0.0,100.65,100.65,20,54


In [13]:
# loop to remove outliers x times
x = 4
n = 1
while n <= x:
    nr_arr = waittimes["SPOSTMIN"].to_numpy()
    indices = []
    for index in range(1,len(nr_arr)):
        if (nr_arr[index] - nr_arr[index-1]) > (waittimes["SPOSTMIN"].std() * 2):
            indices.append(index)
    waittimes = waittimes.drop((indices), axis=0)
    n += 1

waittimes.shape

KeyboardInterrupt: 

In [ ]:
waittimes = imputation(waittimes)

In [10]:
# test set 2021
test = waittimes.loc[waittimes["YEAR"].isin(range(2021,2022))].reset_index(drop=True)
test["YEAR"]


0        2021.0
1        2021.0
2        2021.0
3        2021.0
4        2021.0
          ...  
26868    2021.0
26869    2021.0
26870    2021.0
26871    2021.0
26872    2021.0
Name: YEAR, Length: 26873, dtype: float64

In [11]:
test.isna().sum().sum()

0

In [15]:
# train set 2019-2020
waittimes = waittimes.loc[waittimes["YEAR"].isin(range(2019,2021))].reset_index(drop=True)
waittimes["YEAR"]


0        2019.0
1        2019.0
2        2019.0
3        2019.0
4        2019.0
          ...  
58741    2020.0
58742    2020.0
58743    2020.0
58744    2020.0
58745    2020.0
Name: YEAR, Length: 58746, dtype: float64

In [16]:
waittimes.isna().sum().sum()

0

Spearman's rank correlation (Spearman's Rho)

-> works for non-linear relations, measures the monotonic relation between a pair of variables 

In [17]:
from scipy import stats

X = waittimes.loc[:, waittimes.columns != "SPOSTMIN"]
y = waittimes['SPOSTMIN']

#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
# applying Spearman correlation to data
rho_matrix = X.corr(method="spearman")
# print(rho_matrix)
# prints the rho coefficient for every feature in correlation with another

In [ ]:
# function to get the names of features with a correlation over a chosen threshold
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

# n = chosen threshold
n = 0.95
corr_features = correlation(X, n)
print(str(len(set(corr_features))) + " have a correlation over " + str(n))
#print(corr_features)

In [ ]:
# dropping the features with correlation over the chosen threshold
X.drop(corr_features,axis=1)
#y.drop(corr_features,axis=1)

In [18]:
# applying random forest to selected features
rf = RandomForestRegressor(n_estimators=10, max_depth=50, n_jobs=-1, random_state=42)
rf.fit(X, y)
y_pred = rf.predict(X)

# r2 value is the same as random forest without feature selection
print("RMSE: " + str(round(sqrt(mean_squared_error(y, y_pred)), 2)))
print("R_squared: " + str(round(r2_score(y, y_pred), 2)))

RMSE: 2.81
R_squared: 0.98


Applying model on test data

In [19]:
rfc_base = rf.fit(X, y)

In [20]:
import pickle
with open('rfc_model_pkl', 'wb') as files:
   pickle.dump(rfc_base, files)

In [21]:
with open('rfc_model_pkl' , 'rb') as f:
   rfc_pretrained = pickle.load(f)

In [22]:
X_test = test.loc[:, test.columns != "SPOSTMIN"]
y_test = test['SPOSTMIN']

In [23]:
X_test.drop(corr_features,axis=1)

NameError: name 'corr_features' is not defined

In [24]:
y_pred_pretrained=rfc_pretrained.predict(X_test)

In [25]:
print("RMSE: " + str(round(sqrt(mean_squared_error(y_test, y_pred_pretrained)), 2)))
print("R_squared: " + str(round(r2_score(y_test, y_pred_pretrained), 2)))

RMSE: 13.07
R_squared: 0.33
